In [4]:
import os.path

## Automata files managment

In [2]:
def get_cases(dir='.',reset=False,prefix='semideterministic',cs=[]):
    if not cs or reset:
        cs.clear()
        unique_cases = !autfilt --unique --stats=%f {dir}/{prefix}/*.hoa
        cs.extend([os.path.splitext(os.path.basename(case))[0] for case in unique_cases])
    return cs

In [3]:
def get_complements(case):
    files = !ls complement/{case}*.hoa
    return files

def get_orig_automaton(case):
    return os.path.join("semideterministic",case+".hoa")

In [4]:
def get_algorithm_name(aut_file):
    if aut_file[-4:] != ".hoa":
        return "case"
    division = aut_file[:-4].split("__Complement")
    if len(division) == 1:
        return "input"
    return division[1]

In [ ]:
def get_products(compl):
    prod = os.path.join('products','prod_{}'.format(os.path.basename(compl)))
    comp_prod = os.path.join('products','comp_prod_{}'.format(os.path.basename(compl)))
    return prod,comp_prod

In [9]:
def is_computed(path):
    return os.path.exists(path) and not os.stat(path).st_size == 0

In [ ]:
get_products(get_complements(get_cases()[0])[0])

In [6]:
def get_ltl2dstar_name(aut,orig=True):
    if orig:
        file_n=os.path.basename(aut)
    else:
        file_n=get_simplified_name(aut,'high')
    return os.path.join('LTL2DSTAR',file_n)

In [7]:
def get_ltl2dstar_path(aut):
    return os.path.join('ltl2dstar',get_ltl2dstar_name(aut))

In [8]:
def get_ltl2dstar_name(aut,simplified=False):
    return 'ltl2dstar_'+os.path.basename(aut)

In [9]:
def get_validate_name(case):
    return 'val_{}.hoa'.format(case)

In [10]:
def get_validate_path(case):
    return os.path.join('products',get_validate_name(case))

In [11]:
def par(path):
    '''Return a path enriched with skip to parent directory.'''
    return os.path.join(os.pardir,path)

##  Generation of simplified automata

In [12]:
def get_simplified_path(aut_file,degree='low'):
    ### get the name of the file
    name = get_simplified_name(os.path.basename(aut_file),degree)
    ### add the directory prefix
    return os.path.join('simplified',name)

In [13]:
def get_simplified_name(aut_file,degree="low"):
    return "{}_{}.hoa".format(os.path.basename(aut_file)[0:-4],degree)

In [14]:
def get_simplify_command(aut,output_file,degree='low',timeout='5m'):
    return 'timeout {} autfilt --small --{} -B -H {} > {}'.\
        format(timeout,degree,aut,output_file)

The following definitions are needed only for simplifications here, in ipython. Deprecated, since by Makefile one can use parallelism

## Reading #states

In [15]:
def get_states(aut):
    states=!autfilt --stats=%s {aut}
    if states:
        return states[0]
    else: return ""

In [16]:
def get_and_store_states(aut,entry):
    states = get_states(aut)
    entry[get_algorithm_name(aut)]=states

In [17]:
def extract_states(case):
    entry=dict()
    entry['case']=case
    get_and_store_states(get_orig_automaton(case),entry)
    for aut in get_complements(case)+get_simplified(case)+get_simplified(case,"high"):
        get_and_store_states(aut,entry)
    return entry

In [18]:
def Incrementor(i=0,arg=0):
    while(True):
        yield i
        i+=1
        i+=arg

### Processing data

In [19]:
def collect_states():
    states = []
    ids = Incrementor()
    for case in get_cases():
        results=extract_states(case)
        results['id']=ids.send(None)
        states.append(results)
    return states